# Задача за класификация на текст

Както споменахме, ще се фокусираме върху проста задача за класификация на текст, базирана на набора от данни **AG_NEWS**, която включва класифициране на заглавия на новини в една от 4 категории: Свят, Спорт, Бизнес и Наука/Технологии.

## Наборът от данни

Този набор от данни е вграден в модула [`torchtext`](https://github.com/pytorch/text), така че можем лесно да го достъпим.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Тук `train_dataset` и `test_dataset` съдържат колекции, които връщат двойки от етикет (номер на класа) и текст съответно, например:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

И така, нека отпечатаме първите 10 нови заглавия от нашия набор от данни:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Тъй като наборите от данни са итератори, ако искаме да използваме данните многократно, трябва да ги преобразуваме в списък:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Токенизация

Сега трябва да преобразуваме текста в **числа**, които могат да бъдат представени като тензори. Ако искаме представяне на ниво думи, трябва да направим две неща:
* използваме **токенизатор**, за да разделим текста на **токени**
* изградим **речник** от тези токени.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Използвайки речник, можем лесно да кодираме нашия токенизиран низ в набор от числа:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Представяне на текст чрез "Чанта с думи"

Тъй като думите носят значение, понякога можем да разберем смисъла на текста, като просто разгледаме отделните думи, без да обръщаме внимание на техния ред в изречението. Например, при класифициране на новини, думи като *време*, *сняг* вероятно ще насочат към *прогноза за времето*, докато думи като *акции*, *долар* биха се отнесли към *финансови новини*.

**Чанта с думи** (BoW) е най-често използваното традиционно векторно представяне. Всяка дума е свързана с индекс във вектора, а елементът на вектора съдържа броя на срещанията на дадена дума в конкретен документ.

![Изображение, показващо как представянето чрез "чанта с думи" се съхранява в паметта.](../../../../../translated_images/bg/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Можете също да мислите за BoW като за сума от всички едноразрядно кодирани вектори за отделните думи в текста.

По-долу е даден пример как да генерирате представяне чрез "чанта с думи", използвайки библиотеката Scikit Learn на Python:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

За да изчислим bag-of-words вектор от векторното представяне на нашия AG_NEWS набор от данни, можем да използваме следната функция:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Забележка:** Тук използваме глобалната променлива `vocab_size`, за да зададем стандартния размер на речника. Тъй като често размерът на речника е доста голям, можем да го ограничим до най-често срещаните думи. Опитайте да намалите стойността на `vocab_size` и да изпълните кода по-долу, за да видите как това влияе на точността. Трябва да очаквате известно намаление на точността, но не драстично, за сметка на по-висока производителност.


## Обучение на класификатор с метод Bag-of-Words

Сега, след като научихме как да изградим представяне на текста чрез Bag-of-Words, нека обучим класификатор върху него. Първо, трябва да преобразуваме нашия набор от данни за обучение така, че всички позиционни векторни представяния да бъдат преобразувани в представяне чрез Bag-of-Words. Това може да се постигне, като предадем функцията `bowify` като параметър `collate_fn` на стандартния torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Сега нека дефинираме прост класификаторен невронен модел, който съдържа един линеен слой. Размерът на входния вектор е равен на `vocab_size`, а размерът на изхода съответства на броя на класовете (4). Тъй като решаваме задача за класификация, крайната активационна функция е `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Сега ще дефинираме стандартен цикъл за обучение в PyTorch. Тъй като нашият набор от данни е доста голям, за целите на обучението ще тренираме само за една епоха, а понякога дори за по-малко от една епоха (параметърът `epoch_size` ни позволява да ограничим обучението). Също така ще докладваме натрупаната точност на обучението по време на тренировката; честотата на докладване се определя чрез параметъра `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## Биграми, триграми и N-грами

Едно от ограниченията на подхода с чанта от думи е, че някои думи са част от изрази, състоящи се от няколко думи. Например, думата „hot dog“ има напълно различно значение от думите „hot“ и „dog“ в други контексти. Ако винаги представяме думите „hot“ и „dog“ със същите вектори, това може да обърка модела ни.

За да се справим с това, често се използват **представяния с N-грам** в методите за класификация на документи, където честотата на всяка дума, двойка думи или тройка думи е полезна характеристика за обучение на класификатори. При представяне с биграми, например, ще добавим всички двойки думи към речника, в допълнение към оригиналните думи.

По-долу е даден пример за това как да генерираме представяне с чанта от думи с биграми, използвайки Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Основният недостатък на подхода с N-грам е, че размерът на речника започва да нараства изключително бързо. На практика е необходимо да комбинираме представянето с N-грам с някои техники за намаляване на размерността, като например *вграждания*, които ще обсъдим в следващия модул.

За да използваме представянето с N-грам в нашия **AG News** набор от данни, трябва да изградим специален речник за n-грам:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Можем да използваме същия код като горния, за да обучим класификатора, но това би било много неефективно по отношение на паметта. В следващия модул ще обучим биграм класификатор, използвайки вграждания.

> **Note:** Можете да оставите само онези n-грамове, които се срещат в текста повече от определен брой пъти. Това ще гарантира, че рядко срещаните биграми ще бъдат пропуснати и значително ще намали размерността. За да направите това, задайте параметъра `min_freq` на по-висока стойност и наблюдавайте как се променя дължината на речника.


## Честота на термина и обратна честота на документа (TF-IDF)

В представянето на BoW (Bag of Words), срещанията на думите се претеглят равномерно, независимо от самата дума. Въпреки това е очевидно, че често срещани думи като *а*, *в* и т.н. са много по-малко важни за класификацията в сравнение със специализираните термини. Всъщност, в повечето задачи по обработка на естествен език (NLP) някои думи са по-релевантни от други.

**TF-IDF** означава **честота на термина – обратна честота на документа**. Това е вариация на модела "чанта с думи" (BoW), при която вместо бинарна стойност 0/1, указваща наличието на дума в даден документ, се използва стойност с плаваща запетая, която е свързана с честотата на срещане на думата в корпуса.

По-формално, теглото $w_{ij}$ на дума $i$ в документ $j$ се дефинира като:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
където:
* $tf_{ij}$ е броят на срещанията на $i$ в $j$, т.е. стойността на BoW, която видяхме по-рано
* $N$ е броят на документите в колекцията
* $df_i$ е броят на документите, съдържащи думата $i$ в цялата колекция

Стойността на TF-IDF $w_{ij}$ нараства пропорционално на броя пъти, в които една дума се появява в даден документ, и се коригира спрямо броя на документите в корпуса, които съдържат тази дума. Това помага да се компенсира фактът, че някои думи се срещат по-често от други. Например, ако дадена дума се появява *във всеки* документ от колекцията, тогава $df_i=N$, и $w_{ij}=0$, като тези термини ще бъдат напълно игнорирани.

Можете лесно да създадете векторизация на текст с TF-IDF, използвайки Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Заключение

Въпреки че представянията чрез TF-IDF предоставят тегло на честотата за различни думи, те не могат да представят значението или реда. Както известният лингвист Дж. Р. Фърт каза през 1935 г., „Пълното значение на една дума винаги е контекстуално, и никакво изследване на значението извън контекста не може да бъде взето на сериозно.“. По-късно в курса ще научим как да улавяме контекстуална информация от текст чрез моделиране на езика.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
